In [1]:
import torch
import torchvision as tv
import os
from transformer_flow import Model
import utils
import pathlib
from utils import sqa_save

os.environ["CUDA_VISIBLE_DEVICES"] = "4" # set GPU

utils.set_random_seed(100)
notebook_output_path = pathlib.Path('runs/notebook')

In [ ]:
dataset = 'mnist'
num_classes = 10
img_size = 28
channel_size = 1

# we use a small model for fast demonstration, increase the model size for better results
patch_size = 4
channels = 128
blocks = 4
layers_per_block = 4
# try different noise levels to see its effect
noise_std = 0.1

batch_size = 256
lr = 2e-3
# increase epochs for better results
epochs = 100
sample_freq = 10

if torch.cuda.is_available():
    device = 'cuda' 
elif torch.backends.mps.is_available():
    device = 'mps' # if on mac
else:
    device = 'cpu' # if mps not available
print(f'using device {device}')

fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channel_size * patch_size ** 2, device=device)
fixed_y = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, 10).flatten()

transform = tv.transforms.Compose([
    tv.transforms.Resize((img_size, img_size)),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.5,), (0.5,))
])
data = tv.datasets.MNIST('.', transform=transform, train=True, download=True)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)

model = Model(in_channels=channel_size, img_size=img_size, patch_size=patch_size, 
              channels=channels, num_blocks=blocks, layers_per_block=layers_per_block,
              num_classes=num_classes).to(device)

optimizer = torch.optim.AdamW(model.parameters(), betas=(0.9, 0.95), lr=lr, weight_decay=1e-4)
lr_schedule = utils.CosineLRSchedule(optimizer, len(data_loader), epochs * len(data_loader), 1e-6, lr)

model_name = f'bn'
sample_dir = notebook_output_path / f'{dataset}_samples_{model_name}'
ckpt_file = notebook_output_path / f'{dataset}_model_{model_name}.pth'
sample_dir.mkdir(exist_ok=True, parents=True)

for epoch in range(epochs):
    losses = 0
    for x, y in data_loader:
        x = x.to(device)
        eps = noise_std * torch.randn_like(x)
        x = x + eps
        y = y.to(device)
        optimizer.zero_grad()
        z, outputs, logdets = model(x, y)
        loss = model.get_loss(z, logdets)
        loss.backward()
        optimizer.step()
        lr_schedule.step()
        losses += loss.item()

    print(f"epoch {epoch} lr {optimizer.param_groups[0]['lr']:.6f} loss {losses / len(data_loader):.4f}")
    print('layer norms', ' '.join([f'{z.pow(2).mean():.4f}' for z in outputs]))
    print(f'logdet: {logdets.mean():.4f}, prior p: {0.5 * z.pow(2).mean():.4f}')
    if (epoch + 1) % sample_freq == 0:
        with torch.no_grad():
            samples = model.reverse(fixed_noise, fixed_y)
        sqa_save(samples, sample_dir / f'samples_{epoch:03d}.png')
        latents = model.unpatchify(z[:100])
        sqa_save(latents, sample_dir / f'latent_{epoch:03d}.png')
        print(f'sampling complete. Sample mean: {samples.mean():.4f}, std: {samples.std():.4f}, max: {samples.max():.4f}, min: {samples.min():.4f}')
        print(f'latent mean: {latents.mean():.4f}, std: {latents.std():.4f}')
    print('\n')
torch.save(model.state_dict(), ckpt_file)

using device cuda
Number of parameters: 3.23M
epoch 0 lr 0.002000 loss -1.2315
layer norms 1.0803 0.9744 1.7220 0.9654 1.8649 0.9933 0.9970
logdet: 1.9110, prior p: 0.4985


epoch 1 lr 0.001999 loss -1.4687
layer norms 0.7003 0.9187 1.1680 0.8880 1.1711 0.9636 0.9945
logdet: 2.0153, prior p: 0.4972


epoch 2 lr 0.001998 loss -1.5286
layer norms 0.4520 0.8815 1.0381 0.8485 0.9112 0.9496 0.9825
logdet: 2.0205, prior p: 0.4913


epoch 3 lr 0.001995 loss -1.5484
layer norms 0.3385 0.8569 0.9959 0.8286 0.8401 0.9461 1.0065
logdet: 2.0567, prior p: 0.5033


epoch 4 lr 0.001992 loss -1.5584
layer norms 0.2651 0.8396 0.9473 0.8166 0.7817 0.9476 0.9919
logdet: 2.0473, prior p: 0.4960


epoch 5 lr 0.001987 loss -1.5654
layer norms 0.2167 0.8273 0.8848 0.8100 0.7436 0.9513 0.9955
logdet: 2.0551, prior p: 0.4977


epoch 6 lr 0.001982 loss -1.5705
layer norms 0.1755 0.8177 0.8742 0.8071 0.7039 0.9572 0.9942
logdet: 2.0717, prior p: 0.4971


epoch 7 lr 0.001975 loss -1.5744
layer norms 0.1489 0.8109

In [ ]:
# now we can also evaluate the model by turning it into a classifier with Bayes rule, p(y|x) = p(y)p(x|y)/p(x)
data = tv.datasets.MNIST('.', transform=transform, train=False, download=False)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=False)
num_correct = 0
num_examples = 0
for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)
    eps = noise_std * torch.randn_like(x)
    x = x.repeat(num_classes, 1, 1, 1)
    y_ = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, y.size(0)).flatten()
    with torch.no_grad():
        z, outputs, logdets = model(x, y_)
        losses = 0.5 * z.pow(2).mean(dim=[1, 2]) - logdets # keep the batch dimension
        pred = losses.reshape(num_classes, y.size(0)).argmin(dim=0)
    num_correct += (pred == y).sum()
    num_examples += y.size(0)
print(f'Accuracy %{100 * num_correct / num_examples:.2f}')

Accuracy %98.35


In [ ]:
# import torch
# import os
# from transformer_flow import Model
# import utils

# os.environ["CUDA_VISIBLE_DEVICES"] = "4" # set GPU
# utils.set_random_seed(100)

# num_classes = 10
# img_size = 28
# channel_size = 1

# # we use a small model for fast demonstration, increase the model size for better results
# patch_size = 4
# channels = 128
# blocks = 4
# layers_per_block = 4
# # try different noise levels to see its effect
# noise_std = 0.1

# if torch.cuda.is_available():
#     device = 'cuda' 
# elif torch.backends.mps.is_available():
#     device = 'mps' # if on mac
# else:
#     device = 'cpu' # if mps not available
# print(f'using device {device}')

# fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channel_size * patch_size ** 2, device=device)
# # fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channels, device=device)
# fixed_y = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, 10).flatten()

# # load the model
# model = Model(in_channels=channel_size, img_size=img_size, patch_size=patch_size, 
#               channels=channels, num_blocks=blocks, layers_per_block=layers_per_block,
#               num_classes=num_classes).to(device)
# model.load_state_dict(torch.load("runs/notebook/mnist_model_4_128_4_4_0.10_linears_with_residual.pth"))
# model.eval()

# with torch.no_grad():
#     samples = model.reverse(fixed_noise, fixed_y)
#     # print the mean and std of the samples
#     mean = samples.mean(dim=[0, 2, 3])
#     std = samples.std(dim=[0, 2, 3])
#     print(f'mean: {mean}, std: {std}')